[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Item2vec.ipynb)

# Item2vec

In [ ]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [2]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# 因子数
factors = 100
# エポック数
n_epochs = 30
# windowサイズ
window = 100
# スキップグラム
use_skip_gram = 1
# 階層的ソフトマックス
use_hierarchial_softmax = 0
# 使用する単語の出現回数のしきい値
min_count = 5

In [6]:
# item2vecのインプットに使うデータの生成
item2vec_data = []
movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    # 評価された順に並び替える
    # item2vecではwindowというパラメータがあり、itemの評価された順番も重要な要素となる
    item2vec_data.append(data.sort_values("timestamp")["movie_id"].tolist())


In [ ]:
!pip install gensim==4.0.1

In [8]:
import gensim

# item2vecの学習
model = gensim.models.word2vec.Word2Vec(
    item2vec_data,
    vector_size=factors,
    window=window,
    sg=use_skip_gram,
    hs=use_hierarchial_softmax,
    epochs=n_epochs,
    min_count=min_count,
)

/Users/mk/.pyenv/versions/3.7.8/envs/recsys-test3.7.8/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [19]:
# スターウォーズ・エピーソード５(movie_id=1196）を入力したときの類似映画
# スターウォーズのエピーソード4,6が上位にきているので、学習されていることが分かる
for movie_id, score in model.wv.most_similar(1196):
    title = movies[movies.movie_id == movie_id].title.tolist()[0]
    print(f'movie_id={movie_id}, title={title}, score={score}')

movie_id=1210, title=Star Wars: Episode VI - Return of the Jedi (1983), score=0.8640398979187012
movie_id=260, title=Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977), score=0.8317610025405884
movie_id=1198, title=Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981), score=0.8280032873153687
movie_id=2571, title=Matrix, The (1999), score=0.8175885677337646
movie_id=1240, title=Terminator, The (1984), score=0.7891822457313538
movie_id=1197, title=Princess Bride, The (1987), score=0.7843142747879028
movie_id=1291, title=Indiana Jones and the Last Crusade (1989), score=0.7821475267410278
movie_id=1270, title=Back to the Future (1985), score=0.7789417505264282
movie_id=1200, title=Aliens (1986), score=0.7704472541809082
movie_id=1214, title=Alien (1979), score=0.7694951891899109


In [20]:
# 各ユーザーにレコメンドリストの作成

pred_user2items = dict()
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    input_data = []
    for item_id in data.sort_values("timestamp")["movie_id"].tolist():
        if item_id in model.wv.key_to_index:
            input_data.append(item_id)
    if len(input_data) == 0:
        # おすすめ計算できない場合は空配列
        pred_user2items[user_id] = []
        continue
    recommended_items = model.wv.most_similar(input_data, topn=10)
    pred_user2items[user_id] = [d[0] for d in recommended_items]

pred_user2items

{1: [380, 110, 150, 457, 592, 165, 595, 225, 10, 590],
 2: [480, 1196, 589, 527, 593, 356, 318, 457, 296, 592],
 3: [593, 1196, 2571, 318, 527, 2028, 1198, 4993, 2959, 3578],
 4: [380, 356, 377, 457, 318, 593, 10, 539, 349, 527],
 5: [1193, 1208, 36, 296, 318, 904, 750, 1213, 1288, 908],
 6: [1240, 593, 1210, 2858, 1291, 1270, 1617, 541, 1036, 1214],
 7: [750, 1208, 858, 1193, 1267, 1204, 922, 2019, 1247, 1221],
 8: [1210, 4993, 110, 5952, 296, 2028, 260, 356, 480, 2716],
 9: [1136, 1089, 1617, 296, 593, 2571, 750, 608, 1196, 1265],
 10: [608, 1247, 1207, 318, 1230, 1271, 300, 593, 150, 1304],
 11: [1197, 1036, 589, 1214, 1200, 457, 1265, 356, 110, 919],
 12: [2571, 1196, 1240, 1580, 1291, 2028, 1270, 1210, 589, 1036],
 13: [1196, 1270, 2571, 593, 1240, 356, 1198, 260, 1200, 608],
 14: [4993, 4886, 7153, 1198, 5952, 4306, 2571, 1291, 919, 356],
 16: [541, 2571, 1270, 589, 1198, 1197, 750, 1291, 1374, 1036],
 17: [1196, 904, 1617, 750, 1208, 608, 1240, 912, 2571, 541],
 18: [1197, 1210,

In [21]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [23]:
# user_id=2に対するおすすめ(480, 1196, 589)
movies[movies.movie_id.isin([480, 1196, 589])]

,movie_id,title,genre,tag
476,480,Jurassic Park (1993),"[Action, Adventure, Sci-Fi, Thriller]","[based on a book, biology, michael crichton, s..."
583,589,Terminator 2: Judgment Day (1991),"[Action, Sci-Fi]","[action, sci-fi, dvd, seen more than once, tim..."
1171,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i..."


In [ ]:
# item2vecでは、因子数,エポック数,windowサイズ,使用する単語の出現回数のしきい値のどれもが重要なので、グリッドサーチしながら、最適な値を決めてください